## Mounting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import os

In [0]:
# zmiana working directory
os.chdir("/content/gdrive/My Drive/Data/Elo/")

In [0]:
# sprawdzenie procesora ale jak to czytać?
!cat /proc/cpuinfo

In [0]:
#sprawdzenie ramu, ale jak to czytać?
!cat /proc/meminfo

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c elo-merchant-category-recommendation -p /content/gdrive/My\ Drive/Data

/bin/bash: kaggle: command not found


In [0]:
merchants_types = {
 'merchant_group_id': 'uint32',
 'merchant_category_id':'int16',
 'city_id': 'int16',
 'state_id': 'int8',
 'subsector_id': 'int8',
 'numerical_1': 'float16',
 'numerical_2': 'float16',
 'category_1': 'category', #bool_,
 'category_2': 'category',
 'category_4': 'category',   
 'most_recent_sales_range': 'category',
 'most_recent_purchases_range': 'category',
 'avg_sales_lag3': 'float32',
 'avg_purchases_lag3': 'float32',
 'active_months_lag3': 'float16',
 'avg_sales_lag6': 'float32',
 'avg_purchases_lag6': 'float32',
 'active_months_lag6': 'float16',
 'avg_sales_lag12': 'float32',
 'avg_purchases_lag12': 'float32',
 'active_months_lag12': 'float16'
 }
types_new_merchant_transactions ={
    'card_id':'object',
    'authorized_flag':'category',
    'city_id': 'int16',
    'state_id': 'int8',    
    'subsector_id': 'int8',
    'category_1': 'category',
    'category_2': 'category',
    'category_3': 'category',
    'installments': 'int16',
    'merchant_category_id': 'int16',
    'month_lag': 'uint8', #1,2 
    'purchase_amount': 'float16',
    'purchase_date': 'object'}

In [0]:
#merchants = pd.read_csv("merchants.csv", index_col='merchant_id', dtype = merchants_types)
#new_merchant_transactions = pd.read_csv("new_merchant_transactions.csv", index_col = 'merchant_id', dtype=types_new_merchant_transactions)
hist_trans = pd.read_csv("historical_transactions.csv")
#train = pd.read_csv("train.csv")

In [117]:
new_merchant_transactions.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1963031 entries, M_ID_b0c793002c to M_ID_d855771cd9
Data columns (total 13 columns):
authorized_flag         category
card_id                 object
city_id                 int16
category_1              category
installments            int16
category_3              category
merchant_category_id    int16
month_lag               uint8
purchase_amount         float16
purchase_date           object
category_2              category
state_id                int8
subsector_id            int8
dtypes: category(4), float16(1), int16(3), int8(2), object(2), uint8(1)
memory usage: 73.0+ MB


In [57]:
np.finfo(np.float16).max

65500.0

In [0]:
new_merchant_transactions['purchase_date'] = pd.to_datetime(new_merchant_transactions['purchase_date'], unit='ns')

In [124]:
new_merchant_transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1963031 entries, M_ID_b0c793002c to M_ID_d855771cd9
Data columns (total 13 columns):
authorized_flag         category
card_id                 object
city_id                 int16
category_1              category
installments            int16
category_3              category
merchant_category_id    int16
month_lag               uint8
purchase_amount         float16
purchase_date           datetime64[ns]
category_2              category
state_id                int8
subsector_id            int8
dtypes: category(4), datetime64[ns](1), float16(1), int16(3), int8(2), object(1), uint8(1)
memory usage: 113.0+ MB


In [0]:
types_new_merchant_transactions ={
    'card_id':'object',
    'authorized_flag':'category',
    'city_id': 'int16',
    'state_id': 'int8',    
    'subsector_id': 'int8',
    'category_1': 'category',
    'category_2': 'category',
    'category_3': 'category',
    'installments': 'int16',
    'merchant_category_id': 'int16',
    'month_lag': 'uint8', #1,2 
    'purchase_amount': 'float16',
    'purchase_date': 'object'}

In [0]:
hist_trans.shape

(29112361, 14)

In [0]:
train.shape

(201917, 6)